In [1]:
# Usage of NVDB data
# https://www.vegvesen.no/nvdb/apidokumentasjon

In [21]:
# Notebook config, enviroment and logging

import os
import sys
from datetime import datetime

import logging
from dotenv import load_dotenv

import pakkenellik.config as conf
from pakkenellik.log import logger
from pakkenellik.dataframe.clean_column_headers import clean_column_headers
from pakkenellik.dataframe.regions import add_municipality_name
from pakkenellik.dataframe.datetime import add_ymd, add_month_name, add_week_day, add_week_day_name

# Add module path and load config
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Autoreload extension
if "autoreload" not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Load the .env file into local env
load_dotenv()

# Create the config
config = conf.Config(module_path)

# Enable logging and crank up log level to DEBUG.
# This is particularly useful when developing code in your project module and using it from a notebook.
logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# nvdbapiv3 and Geopandas
import pandas as pd
import geopandas as gpd
import nvdbapiv3

In [8]:
ras_data = nvdbapiv3.nvdbFagdata(445)
df = pd.DataFrame(ras_data.to_records())
df = clean_column_headers(df)
df.head()

Eksport av 59979 objekter kommer til å ta tid...
Objekt 1000 av 59979
Objekt 5000 av 59979
Objekt 10000 av 59979
Objekt 20000 av 59979
Objekt 30000 av 59979
Objekt 40000 av 59979
Objekt 50000 av 59979


,objekttype,nvdbid,versjon,startdato,skred_dato,skred_klokkeslett,type_skred,volum_av_skredmasser_paa_veg,losneomraade,vesentlige_skader,...,utgaar_vindretning_ved_vaerstasjon,groftebredde_paa_skredside,utgaar_mengde_sno,utgaar_mengde_regn,utgaar_vindstyrke_v_vaerstasjon,tilleggsinformasjon,registreringsansvarlig,utgaar_mengde_vaat_snosludd,antatt_hovedaarsak,prosjektreferanse
0,445,78774366,1,2006-02-14,2006-02-14,1700.0,Stein,< 1 kbm,Fjell/dalside,Nei,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,445,78774368,1,2006-01-31,2006-01-31,2200.0,Is,<10 kbm,Vegskjæring,Nei,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,445,78774370,1,2006-02-06,2006-02-06,630.0,Stein,< 1 kbm,Vegskjæring,Nei,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,445,78774372,1,2006-02-06,2006-02-06,630.0,Stein,<10 kbm,Vegskjæring,Nei,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,445,78774374,1,2006-02-16,2006-02-16,1100.0,Snø,< 1000 kbm,Fjell/dalside,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.columns

Index(['objekttype', 'nvdbid', 'versjon', 'startdato', 'skred_dato',
       'skred_klokkeslett', 'type_skred', 'volum_av_skredmasser_paa_veg',
       'losneomraade', 'vesentlige_skader', 'stedsangivelse',
       'vaerforhold_paa_vegen', 'blokkert_veglengde', 'stengning',
       'utgaar_stengt_fra_dato', 'utgaar_stengt_fra_klokkeslett',
       'utgaar_aapent_fra_dato', 'relasjoner', 'veglenkesekvensid',
       'detaljnivaa', 'typeveg', 'kommune', 'fylke', 'vref', 'veglenketype',
       'vegkategori', 'fase', 'vegnummer', 'startposisjon', 'sluttposisjon',
       'segmentlengde', 'adskilte_lop', 'trafikantgruppe', 'geometri',
       'hoydeforskjell_vegutlosningssted', 'utgaar_aapent_fra_klokkeslett',
       'skade_paa_kjoretoy', 'skade_paa_vegdekkevegkropp',
       'temperatur_paa_veg', 'vindretning', 'skade_paa_drenering',
       'skade_paa_rekkverk', 'skade_paa_bru', 'bilder',
       'utgaar_nedbor_siste_3_dogn', 'volum_av_totale_skredmasser',
       'skade_paa_person', 'utgaar_temperat

In [14]:
df.skred_dato = pd.to_datetime(df.skred_dato, format='%Y-%m-%d')
df = add_ymd(df, "skred_dato")
df = add_month_name(df, "skred_dato")
df = add_week_day(df, "skred_dato")
df = add_week_day_name(df, "skred_dato", nynorsk=True)
df = add_municipality_name(df, "kommune")

In [27]:
gs = gpd.GeoSeries.from_wkt(df["geometri"])
gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

In [28]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# using to_file with drive=GeoJSON saves a json. This saves a geojson
with open(config.get_processed_file(f"etl_e16_{timestamp}.geojson"), 'w') as f:
    # datetime don't json, so set default str
    f.write(gdf[(gdf.vegkategori == "E") & (gdf.vegnummer == 16)][["skred_dato", "geometry"]].to_json(default=str))

In [34]:
df.to_csv(config.get_processed_file("etl_e16.csv"), index=False)